# 데이터 이해하기

In [10]:
import pandas as pd
import os

In [17]:
path=os.getenv("HOME") + ('/Documents/LPOINT/data/')

In [21]:
filename = ["online_activity_information.csv", "item_classfication.csv", "customer_demo.csv", "transaction_information.csv"]
activityInfo = pd.read_csv(path + filename[0])
items = pd.read_csv(path + filename[1])
customers = pd.read_csv(path + filename[2])
transactionInfo = pd.read_csv(path + filename[3])

## 온라인 행동 정보

In [26]:
activityInfo.sample(5)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
2041655,2583,1,21,6,A02,20190722,13:41,1524725,57499.0,NaN,46.0,2106.0,unknown,PC
3166375,16207,17,1,0,A02,20190806,21:26,7754,NaN,하이원,3.0,19.0,unknown,mobile_app
2888038,39156,107,8,0,A03,20190926,23:33,1924770,NaN,쿨피스,32.0,3445.0,DIRECT,NaN
1641385,33727,19,3,4,A03,20190803,08:35,13351,NaN,NaN,6.0,45.0,DIRECT,NaN
2121341,17881,12,2,1,A02,20190820,03:48,114362,NaN,NaN,14.0,1091.0,unknown,mobile_app


In [61]:
activityInfo['biz_unit'].unique()

array(['A03', 'A02', 'A01'], dtype=object)

### 해석 예시
client 2583번 사람이, 21번째 조회 때, 6번 행동(구매완료)를 수행했다.biz_unit은 온라인 이용처 A02에서 구매했고, 
2019년 7월 22일 13시 41분에 일어난 일이다.
세션이 시작된 이후 해당(21번째) 조회까지 경과한 시간은 1524725 밀리세컨드이다.
주문번호는 57499임. 행동이 검색이 아니라서 키워드는 없음. 세션이 시작된 후부터 지금까지 총 46개의 페이지 화면을 봤다. 세션 시작부터 지금까지 2106초동안 있었다. 고객 유입채널은 알 수 없었고, 이 사람은 PC를 사용했다.

*총 8가지의 행동 유형을 구분한 코드
[0.검색 / 1.제품 목록 / 2.제품 세부정보 보기 / 3.장바구니 제품 추가 /
4.장바구니 제품 삭제 / 5.결제 시도 / 6.구매 완료 / 7.구매 환불 / 8.결제 옵션 ]

### Questions
1. biz_unit 레이블 의미는? 전부 온라인일까..? 이용처가 뭐지. 지마켓, 11번가 이런 건가..? 오.. 그럼 기본적으로 지금, L point 데이터는.. 뭐지?!!!! 쇼핑몰이 기준이 아니라면 엘포인트에서 판매하고 있는 제품들에 대한 구매 데이터인가?
2. HIT_PSS_TM(조회경과시간) : 세션이 시작된 이후 해당 조회까지 경과한 시간(단위 : 밀리초) ※ 첫 번째 조회이면 0으로 설정됩니다. / TOT_SESS_HR_V(총세션시간값) : ᆞ세션 내 총 시간(단위: 초) 두 항목의 차이는?

### 아이디어 - 데이터로부터 고객의 행동 패턴을 추측할 수 있을 듯
1. 조회 일련 번호, 조회경과시간 등을 통해 고객들이 어떤 패턴을 통해 구매까지 가거나 구매를 하지 않는가 등을 알 수 있을 거 같음. 빠르게 선택해서 구매하는 사람도 있고 오래 머물러서 구매할 수도 있고, 그럴 때 어느 행동을 하는 조회에서 오래 머무는지 등.. 그런 것을 통해 구매전환율을 높이기 위한 아이디어를 찾아낼 수 있을 듯... 그리고 고객의 성향을 분류할 수 있지 않을까. 누구는 빠르게 선택하는 스타일, 누구는 장바구니에 담았다가 뺐다가 구매하는 스타일 등. 그래서, 고객의 온라인 행동 정보 -> 고객의 쇼핑 성향 분류한 것을 가공 데이터로 만들어서 추천 시스템에 고객 demo와 더하여 추천 해보기..?
2. 검색 키워드와 구매 성공까지의 관계. 검색한 키워드에 해당하는 상품만을 사는지, 아니면 검색 없이 사는 것도 있는지. 아마 검색 없이 사는 거면, 추천 상품 목록을 클릭해서 그 상품 페이지로 접속하는 것일까..?
3. PC와 모바일과 구매 성공까지의 관계.
4. 세션 접속 시각(낮, 밤, 점심, 새벽 등), 요일도 구매하는 item에 영향을 미치지 않을까?


### 결론
- 고객이 쇼핑할 때의 행동 패턴, 고객의 쇼핑 성향
- 검색 키워드 및 실제 구매 아이템의 관계
- PC, 모바일에 따라 더 잘 구매되는 아이템이 있을까
- 세션 접속 시각에 따라 더 잘 구매되는 아이템이 있을까

## 상품 분류

In [27]:
items.sample(5)

,pd_c,clac_nm1,clac_nm2,clac_nm3
452,453,Fashion Accessories,Kids' Shoes,Kids' Fashion Sneakers
28,29,Automotive Products,Seats / Cushions / Mats,Car Seats
876,877,Kitchenware,Kitchen Organization / Disposable Goods,Dish Rags
1633,1634,Video / Audio System Electronics,Cameras / Camcorders,DSLR
713,714,Health Foods,Honey,Royal Jelly


In [45]:
items.shape

(1667, 4)

In [47]:
items.nunique(axis=0)

pd_c        1667
clac_nm1      60
clac_nm2     332
clac_nm3    1662
dtype: int64

In [50]:
items[items["clac_nm3"].isnull()]

,pd_c,clac_nm1,clac_nm2,clac_nm3
195,196,Chilled Foods,Packaged Side Dishes,NaN
523,524,Frozen Foods,Frozen Instant Foods,NaN


In [56]:
items[items.duplicated(['clac_nm3'])==True]

,pd_c,clac_nm1,clac_nm2,clac_nm3
523,524,Frozen Foods,Frozen Instant Foods,NaN
1433,1434,Tableware / Cooking Utensils,Vacuum Sealed / Storage Containers,Lunch Boxes
1570,1571,Vegetables,Fruit Vegetables,Peppers
1627,1628,Vegetables,Western Vegetables,Salads


In [58]:
items[items['clac_nm3']=="Lunch Boxes"]

,pd_c,clac_nm1,clac_nm2,clac_nm3
272,273,Cooked Foods,Chilled Snacks,Lunch Boxes
1433,1434,Tableware / Cooking Utensils,Vacuum Sealed / Storage Containers,Lunch Boxes


In [60]:
items[items['clac_nm3']=="Peppers"]

,pd_c,clac_nm1,clac_nm2,clac_nm3
1263,1264,Spices / Seasonings,Spices,Peppers
1570,1571,Vegetables,Fruit Vegetables,Peppers


In [59]:
items[items['clac_nm3']=="Salads"]

,pd_c,clac_nm1,clac_nm2,clac_nm3
274,275,Cooked Foods,Chilled Snacks,Salads
1627,1628,Vegetables,Western Vegetables,Salads


clac1 : 대분류, clac2 : 중분류, clac3 : 소분류
pd_c : 소분류코드

## 고객 demo 정보

In [78]:
customers.sample(5)

,clnt_id,clnt_gender,clnt_age
9651,9657,unknown,unknown
20436,20446,unknown,unknown
34525,34541,F,30
9098,9104,unknown,unknown
20205,20215,unknown,unknown


In [79]:
customers.shape

(72399, 3)

In [76]:
customers[customers['clnt_gender']=="unknown"]

,clnt_id,clnt_gender,clnt_age
0,1,unknown,unknown
2,3,unknown,unknown
3,4,unknown,unknown
4,5,unknown,unknown
5,6,unknown,unknown
...,...,...,...
72394,72425,unknown,unknown
72395,72426,unknown,unknown
72396,72427,unknown,unknown
72397,72428,unknown,unknown


In [77]:
customers[customers['clnt_age']=="unknown"]

,clnt_id,clnt_gender,clnt_age
0,1,unknown,unknown
2,3,unknown,unknown
3,4,unknown,unknown
4,5,unknown,unknown
5,6,unknown,unknown
...,...,...,...
72394,72425,unknown,unknown
72395,72426,unknown,unknown
72396,72427,unknown,unknown
72397,72428,unknown,unknown


unknown인 게 많아서, 어떻게 사용할 수 있을지 고민해 봐야할 듯..

## 거래 정보

In [29]:
transactionInfo.sample(5)

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
108438,33877,58801,9,A03,0529,20190724,15:13,6300,1
94201,36982,75690,1,A03,0521,20190814,16:53,4900,1
358691,65103,7057,2,B02,0348,20190703,10:49,3490,1
148422,2196,47423,15,A03,1183,20190711,22:40,2990,1
91351,25169,75520,5,A03,1184,20190814,12:17,2490,1


### 해석 예시
client 33877번 사람(key)이, 주문번호가 58801인 주문이 있음(key), 
총 9개 종류의 상품을 구매하였고,
A03이라는 온/오프라인 이용처로 구매함, 
상품 분류 코드는 529번인 상품을 샀음. 구매 날짜는 2019년 7월 24일,
15시 13분에 구매를 하였고, 이 상품은 6300원짜리, 한개를 산 거임.

### 아이디어 - 데이터로부터 고객의 행동 패턴을 추측할 수 있을 듯
1. 한 번에 같이 구매하는 아이템끼리의 관계가 있을까? 동일 대분류에 있는 상품이라던지..
2. 무료 배송 기준 금액이 있다면, 그걸 넘기기 위해 추가로 구매하는 것들이 있을텐데, 그런 정보는 구할 수 없겠지?
3. 같은 고객은 서로 다른 거래(주문번호)마다 비슷한 종류의 물품을 사는지, 아니면 다른 물품들을 사는지! 이미 샀으니 또 살 필요가 없어서 안산다 vs 소모품이라 또 필요하게 되어있어서 산다. 등 여러 케이스가 나올 듯. 그 케이스들을 분류해서, 상품 종류들에 따라 재구매 성이 강한 카테고리, 한번 사면 다신 안산다 카테고리 등으로 나눌 수 있지 않을까

### 결론
- 하나의 거래내역 내의 구매 아이템 사이의 관계
- 한 고객의 거래내역들 사이의 구매 아이템 관계

In [62]:
transactionInfo['biz_unit'].unique()

array(['A03', 'B01', 'B02', 'A02', 'A01', 'B03'], dtype=object)

In [67]:
transactionInfo.shape

(599961, 9)

In [66]:
transactionInfo['trans_id'].nunique()

116440

In [69]:
transactionInfo[transactionInfo['trans_id'].isnull()]

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct


In [71]:
transactionInfo[transactionInfo.duplicated(['trans_id'])==True]

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
4,39423,105124,13,A03,0565,20190920,17:26,2990,1
5,21279,104907,1,A03,0565,20190920,10:27,2990,1
11,39423,105124,3,A03,unknown,20190920,17:26,790,1
14,19163,104867,1,A03,0178,20190920,09:55,9900,1
15,18478,104878,1,A03,0183,20190920,10:03,7900,1
...,...,...,...,...,...,...,...,...,...
599956,35311,2839,3,B03,0339,20190927,14:09,3500,1
599957,35311,2839,4,B03,0339,20190927,14:09,3600,2
599958,35311,2839,5,B03,0339,20190927,14:09,3500,1
599959,35311,2839,2,B03,0339,20190927,14:09,4500,1


In [72]:
transactionInfo[transactionInfo['trans_id']==105124]

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1
4,39423,105124,13,A03,0565,20190920,17:26,2990,1
11,39423,105124,3,A03,unknown,20190920,17:26,790,1
136,39423,105124,7,A03,unknown,20190920,17:26,2990,1
263,39423,105124,8,A03,unknown,20190920,17:26,2980,2
268,39423,105124,4,A03,0933,20190920,17:26,2990,1
340,39423,105124,5,A03,0348,20190920,17:26,1990,1
462,39423,105124,12,A03,unknown,20190920,17:26,3590,1
2532,39423,105124,6,A03,unknown,20190920,17:26,4990,1
2548,39423,105124,9,A03,0179,20190920,17:26,9900,1


# 추가 의견

1. 데이터가 총 3개월 어치.. 2개월 간의 데이터로 학습하고 추천했을 때, 해당 상품이 1개월 데이터에 포함되는가로 평가.

=> 그렇게 하려면, 3개월 째에 구매이력이 존재하는 고객으로 학습을 해야함

2. 외부 데이터 : 기존 데이터와 맞는 key가 있어야 하는데, 외부 데이터 중에 일반적일 수 있는 데이터 항목을 고려해서 생각해보았음
- 구매 시각/요일/계절 별 상품분류
- 성별 별 상품분류
- 연령대 별 상품분류
- 보통 함께 구매하는 상품들

# 프로젝트 진행 순서에 대한 의견

### 구조 : 고객 데이터 기반으로 아이템 판매에 영향을 주는 요인들(고객의 특징 + 고객의 과거 구매내력) + 고객과 무관하게 아이템 판매에 영향을 주는 요인들 => 추천 아이템
1. 고객의 온라인 행동 정보 + 고객의 데모그래픽 => 고객의 특징 (쇼핑 성향)
- sequence 를 고려하여 처리해야 함 (RNN 구조)
2. (고객과 무관하게) 시각, 요일, 계절, 세트로 잘 팔리는 아이템들, 접속 device 별 잘 팔리는 아이템, 검색 키워드 별 실제 팔린 아이템 => 아이템 판매에 영향을 주는 요인들 
- 임베딩?